

All Asgard genomes were downloaded and stored in the directory "ncbi-genomes-2021-03-21". All genomes were compiled into a same bgff file called "asgard_assemblies.gbff" by means of a simple bash command: cat *.gbff > asgard_assemblies.gbff

In [1]:
from Bio import SeqIO # To parse the Genbank files (gbff) with SeqIO.parse().
import re
import sys

In [3]:
unique_hits_protein_id = open("crispr_hits_protein_id.txt", "r")
unique_hits_protein_id = unique_hits_protein_id.readlines()
unique_hits_protein_id = [hit[:-1] for hit in unique_hits_protein_id] # Remove the trailing \n.

# Printing features that are in the region around each CRISPR-hit

Print features (usually, gene products) that are around each CRISPR hit. For each hit, 7 features are printed:
The hit itself, 3 features on the 5' side, and 3 features on the 3' side.

First, I create a full-feature version. This version prints all information in each feature: protein ID, protein sequence, product, locus tag, etc. This version is meant to be human readable, that is, in order to manually explore the 7 features around a particular hit.

Then, I create a product version. The "product" field of a feature indicates the automatic prediction of what that gene product is. Examples are "putative ski2-type helicase" or "CRISPR-associated protein, Csh2 family". In the product version of the hit neighborhood, I print only the product field of the features around each hit.

In [ ]:
# Full-feature neighborhood.

asgard_genomes = open("ncbi-genomes-2021-03-21/asgard_assemblies.gbff", "r")
asgard_genomes = SeqIO.parse(asgard_genomes, "genbank")
counter = 0
original_stdout = sys.stdout

with open("neighborhood_hits_full.txt", "w") as neighborhood_hits:
    sys.stdout = neighborhood_hits # Redirect the output generated with print statements to the neighborhood_hits file.
    for record in asgard_genomes:
        for index, feature in enumerate(record.features):
            try:
                if feature.type == "CDS" and feature.qualifiers["protein_id"][0] in unique_hits_protein_id:
                    counter +=1
                    print("Hit Number: ", counter, "Feature index: ", index, "Number of features: ", len(record.features))
                    print("Neighbors 5'") # Printing neighbors on the 5' side.
                    
                    # Every protein has a CDS feature and a gene feature. Therefore, in the following lines, by saying
                    # "range from 1 to 7" I am saying "the features corresponding to the hit and the next 3 proteins".
                    
                    for i in reversed(range(1, min(8, index))): # If there are less than 7 features before the hit,
                        # the index will be smaller than 8, and I will just print all features before the hit.
                        if record.features[index-i].type not in ["gene", "assembly_gap", "source"]: # I want to avoid printing
                            # gene features, because all proteins have a very similar CDS feature, and assembly gaps and the sample source,
                            # because I am not interested in those features, 
                            print(record.features[index-i])
                    print("HIT")        
                    print(feature)
                    print("Neighbors 3'") # Printing neighbors on the 3' side.
                    for i in range(1, min(8, len(record.features))):
                        if record.features[index+i].type not in ["gene", "assembly_gap", "source"]:
                            print(record.features[index+i])             
            except:
                pass
sys.stdout = original_stdout

In [40]:
# Product version of hit neighborhood.

asgard_genomes = open("ncbi-genomes-2021-03-21/asgard_assemblies.gbff", "r")
asgard_genomes = SeqIO.parse(asgard_genomes, "genbank")

with open("neighborhood_hits_products.txt", "w") as neighborhood_hits:
    for record in asgard_genomes:
        for index, feature in enumerate(record.features):
            try:
                if feature.type == "CDS" and feature.qualifiers["protein_id"][0] in unique_hits_protein_id:
                    for i in reversed(range(1, min(8, index))):
                        if record.features[index-i].type not in ["gene", "assembly_gap", "source"]:
                            neighborhood_hits.write(str(record.features[index-i].qualifiers["product"][0]) + "\n")    
                    neighborhood_hits.write(str(feature.qualifiers["product"][0]) + "\n")
                    for i in range(1, min(8, len(record.features))):
                        if record.features[index+i].type not in ["gene", "assembly_gap", "source"]:
                             neighborhood_hits.write(str(record.features[index+i].qualifiers["product"][0]) + "\n")             
            except:
                pass

NameError: name 'original_stdout' is not defined

# Printing the DNA sequence around each hit

As CRISPR arrays are expected to be in a zone nearby CRISPR-Cas systems, the 20000 bp around each hit (10000 bp on each side) were analyzed. Those 20000 bp around each hit are stored in the dnaseq_hits.fasta file, each header being the Genbank ID of the hit.

In [5]:
asgard_genomes = open("ncbi-genomes-2021-03-21/asgard_assemblies.gbff", "r")
asgard_genomes = SeqIO.parse(asgard_genomes, "genbank")

with open("dnaseq_hits.fasta", "w") as dnaseq_hits: 
    for record in asgard_genomes:
        for feature in record.features:
            try:
                if feature.type == "CDS" and feature.qualifiers["protein_id"][0] in unique_hits_protein_id:
                    start = feature.location.start
                    end = feature.location.end
                    dnaseq_hits.write(">" + feature.qualifiers["protein_id"][0] + "\n")
                    window_10000 = record.seq[start - min(10000, start): end + min(10000, len(record.seq)-end)]
                    # window_10000 represents a window of 10000 bp on each side of the hit.
                    if feature.location.strand == -1:
                        window_10000 = window_10000.reverse_complement()[::-1] # Take the reverse complementary, and reverse it, to get the complementary.
                    dnaseq_hits.write(str(window_10000) + "\n")
            except:
                pass